# Сегментация УЗИ снимков

Отладка на слабом железе

In [1]:
import os
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import cv2

from matplotlib import pyplot as plt

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
DATA_PATH = 'data/'
MODEL_PATH = 'models/'

data_shape = (420, 580)

# Mean and Std for image pixels
img_mean = (0.38983212684516944,)
img_std = (0.21706658034222048,)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Helping functions such as run length encoding to genetate submission file

In [4]:
def rle_encoding(x):
    """
    x: numpy array of shape (height, width), 1 - mask, 0 - background
    Returns run length as list
    source: https://www.kaggle.com/rakhlin/fast-run-length-encoding-python
    """
    dots = np.where(x.T.flatten()==1)[0] # .T sets Fortran order down-then-right
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b+1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

Dataset class to get images and masks.  


In [5]:
# data transform
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(img_mean, img_std)
            ])


In [6]:
class USIDataset(Dataset):
    """
    Ultrasaund Nerve Segmentation dataset for PyTorch
    """
    
    def __init__(self, data_dir, train=True, transform=None):
        """
        data_dir (string): path to dir with images
        train (bool): if True aso returns image and mask image.
            If False returns image and mask filename. Should be False for
            test dataset
        transform (callable): Optional transform to be applied on sample
        """
        self.data_dir = data_dir
        self.train = train
        self.transform = transform
        
        self.img_filelist = [ii for ii in os.listdir(data_dir) if 'mask' not in ii]
        self.img_filelist.sort(key=lambda fname: (int(fname.split('_')[0]), int(fname.split('_')[1].split('.')[0])))
        
    def __len__(self):
        return len(self.img_filelist)
    
    def __getitem__(self, idx):
        img_fname = self.img_filelist[idx]
        mask_fname = img_fname.replace('.tif', '_mask.tif')
        
        img = cv2.imread(os.path.join(self.data_dir, img_fname), cv2.IMREAD_GRAYSCALE)
        img = np.expand_dims(img, axis=2)
        
        if self.train:
            mask = cv2.imread(os.path.join(self.data_dir, mask_fname), cv2.IMREAD_GRAYSCALE)
            mask = np.expand_dims(mask, axis=2)
        else:
            mask = mask_fname
        
        if self.transform:
            img = self.transform(img)
            if self.train:
                mask = self.transform(mask)
            
        return (img, mask)

In [7]:
class ConvBlock(nn.Module):
    """
    3x3 conv -> ReLu -> 3x3 conv -> ReLu
    """
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        
        self.conv1 = nn.Conv2d(self.in_channels, self.out_channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(self.out_channels, self.out_channels, kernel_size=3, padding=1)
        
        nn.init.kaiming_normal_(self.conv1.weight)
        nn.init.kaiming_normal_(self.conv2.weight)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        
        return x

In [8]:
class UNet3(nn.Module):
    def __init__(self, n_filters=64):
        super(UNet3, self).__init__()
        
        self.n_filters = n_filters
        
        # convolution blocks
        self.cb_d1 = ConvBlock(1, self.n_filters)
        self.cb_d2 = ConvBlock(self.n_filters, self.n_filters * 2)
        self.cb_bottom = ConvBlock(self.n_filters * 2, self.n_filters * 4) # bottom block
        self.cb_u1 = ConvBlock(self.n_filters * 4, self.n_filters * 2)
        self.cb_u2 = ConvBlock(self.n_filters * 2, self.n_filters)
        
        # max pool
        self.mp1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.mp2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # up conv
        self.uc1 = nn.ConvTranspose2d(self.n_filters * 4, self.n_filters * 2, kernel_size=2, stride=2)
        self.uc2 = nn.ConvTranspose2d(self.n_filters * 2, self.n_filters, kernel_size=2, stride=2)
        
        # 1x1 convolution to get 1 channel image
        self.conv1x1 = nn.Conv2d(self.n_filters, 1, kernel_size=1)
        
        nn.init.kaiming_normal_(self.uc1.weight)
        nn.init.kaiming_normal_(self.uc1.weight)
        nn.init.kaiming_normal_(self.conv1x1.weight)
        
    def forward(self, x):
        # go down
        out1 = self.cb_d1(x)
        x = self.mp1(out1)
        
        out2 = self.cb_d2(x)
        x = self.mp2(out2)
        
        # bottom block
        x = self.cb_bottom(x)
        
        # go up
        x = self.uc1(x)
        x = torch.cat([out2, x], dim=1)
        x = self.cb_u1(x)
        x = self.uc2(x)
        x = torch.cat([out1, x], dim=1)
        x = self.cb_u2(x)
        
        # 1x1 convolution
        x = self.conv1x1(x)
        
        return F.softmax(x)

In [9]:
def logdyson_loss(prediction, true):
    eps = 1e-10
    return torch.mean(
        torch.log( 2 - ((prediction * true).mean(dim=[-1, -2]) + eps) / 
                  ((prediction + true).mean(dim=[-1, -2]) + eps)))

In [10]:
def check_accuracy(loader, model):
    model.eval()
    
    with torch.no_grad():
        for x, y in loader.values():
            x = x.to(device)
            y = y.to(device)
            
            prediction = model(x)

In [11]:
def train_unet(model, optimizer, dataloader, epochs=1, print_every=10):
    for ee in range(epochs):
        for nn, (x, y) in enumerate(dataloader):
            model.train()
            x = x.to(device=device)
            y = y.to(device=device)
            
            scores = model(x)
            prediction = F.softmax(scores)
            loss = logdyson_loss(prediction, y)
            
            optimizer.zero_grad()
            
            loss.backward()
            
            optimizer.step()
            
            #if nn % print_every == 0:
            #    print('Iteration %d, loss = %.4f' % (nn, loss.item()))
            #    check_accuracy(loader_val, model)
            #    print()

In [12]:
params = {
    'batch_size': 10,
    'learning_rate': 1e-2,
    'momentum': 0.9,
    'n_filters': 32
}

NUM_TRAIN = 4508 # full dataset has 5635 points
#NUM_TRAIN = 101 # for dev purposes

model = UNet3(n_filters=params['n_filters']).to(device)

optimizer = torch.optim.SGD(model.parameters(), 
                            lr=params['learning_rate'], 
                            momentum=params['momentum'], 
                            nesterov=True)

dataset = USIDataset(os.path.join(DATA_PATH, 'train'), train=True, transform=transform)

dataloader_full = DataLoader(dataset, batch_size=params['batch_size'], shuffle=True)

dataloader_train = DataLoader(dataset, 
                              batch_size=params['batch_size'], 
                              sampler=torch.utils.data.sampler.SubsetRandomSampler(range(NUM_TRAIN)))

dataloader_val = DataLoader(dataset, 
                            batch_size=params['batch_size'], 
                            sampler=torch.utils.data.sampler.SubsetRandomSampler(range(NUM_TRAIN, len(dataset))))

In [13]:
%%time
train_unet(model, optimizer, dataloader_train, epochs=1, print_every=100)

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:51: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


CPU times: user 2min 9s, sys: 46.3 s, total: 2min 55s
Wall time: 2min 35s


In [16]:
#torch.save(model, os.path.join(MODEL_PATH, 'model.pt'))

In [15]:
print(model)

UNet3(
  (cb_d1): ConvBlock(
    (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (cb_d2): ConvBlock(
    (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (cb_bottom): ConvBlock(
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (cb_u1): ConvBlock(
    (conv1): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (cb_u2): ConvBlock(
    (conv1): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (mp1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1,